# 第8章　値を比較するグラフ

## 8.1 棒グラフとヒートマップ

In [1]:
import json
import pandas as pd
import seaborn as sns

from plotly import graph_objects as go
from plotly import express as px
from plotly.graph_objs.layout import Template

# HealthexpデータセットのDataFrameを読み込み
df = sns.load_dataset('healthexp')

df

,Year,Country,Spending_USD,Life_Expectancy
0,1970,Germany,252.311,70.6
1,1970,France,192.143,72.2
2,1970,Great Britain,123.993,71.9
3,1970,Japan,150.437,72.0
4,1970,USA,326.961,70.9
...,...,...,...,...
269,2020,Germany,6938.983,81.1
270,2020,France,5468.418,82.3
271,2020,Great Britain,5018.700,80.4
272,2020,Japan,4665.641,84.7


In [2]:
# 変数「Year」で分割したGroupを作成
gb = df.groupby('Year')

# たとえば1990年のグループ
df_subset = gb.get_group(1990)

df_subset

,Year,Country,Spending_USD,Life_Expectancy
89,1990,Canada,1699.774,77.3
90,1990,Germany,1724.332,77.3
91,1990,France,1459.110,77.0
92,1990,Great Britain,782.612,75.7
93,1990,Japan,1088.959,78.9
94,1990,USA,2684.984,75.3


In [3]:
# Traceのlistを作成
traces = []
for year in [2018, 2019, 2020]:
    df_subset = gb.get_group(year)
    
    trace = go.Bar(
        x=df_subset['Country'],         # x軸の変数
        y=df_subset['Spending_USD'],    # y軸の変数
        name=year                       # Trace名
    )   # 国別の医療費の棒グラフ
    traces.append(trace)

traces

[Bar({
     'name': '2018',
     'x': array(['Canada', 'Germany', 'France', 'Great Britain', 'Japan', 'USA'],
                dtype=object),
     'y': array([ 5308.356,  6281.84 ,  5099.306,  4189.708,  4554.276, 10451.386])
 }),
 Bar({
     'name': '2019',
     'x': array(['Canada', 'Germany', 'France', 'Great Britain', 'Japan', 'USA'],
                dtype=object),
     'y': array([ 5189.721,  6407.928,  5167.839,  4385.463,  4610.794, 10855.517])
 }),
 Bar({
     'name': '2020',
     'x': array(['Canada', 'Germany', 'France', 'Great Britain', 'Japan', 'USA'],
                dtype=object),
     'y': array([ 5828.324,  6938.983,  5468.418,  5018.7  ,  4665.641, 11859.179])
 })]

In [4]:
# 独自テンプレートを読み込み
with open('custom_white.json') as f:
    custom_white_dict = json.load(f)
    template = Template(custom_white_dict)

# Layoutを作成
layout = go.Layout(
    template=template,                      # グラフテンプレート
    title={'text': 'Healthexp dataset'},    # グラフタイトル
    xaxis={'title': 'Country'},             # x軸ラベル
    yaxis={'title': 'Spending USD [$]'}     # y軸ラベル
)

# Figureを作成
figure = go.Figure(data=traces, layout=layout)

figure

In [5]:
# 積み上げ棒グラフのLayoutを作成
# 作成済みのFigureで figure.update_layout(barmode='stack') でも可能
layout = go.Layout(
    template=template,
    title={'text': 'Healthexp dataset'},
    xaxis={'title': 'Country'},
    yaxis={'title': 'Spending USD [$]'},
    barmode='stack'     # 棒グラフ種類（積み上げ棒グラフ）
)

# Figureを作成
figure = go.Figure(data=traces, layout=layout)

figure

In [6]:
# Traceのlistを作成
traces = []
for year in [1980, 2000, 2020]:
    df_subset = gb.get_group(year)

    trace = go.Bar(
        x=df_subset['Life_Expectancy'],
        y=df_subset['Country'],
        customdata=df_subset['Spending_USD'],   # ホバー表示用の変数
        hovertemplate='Life expectancy: %{x}<br>Spending: $%{customdata}',
        name=year,
        orientation='h'                         #　棒グラフ方向（横棒グラフ）
    )   # 国別の平均余命の棒グラフ
    traces.append(trace)

# Layoutを作成作成
layout = go.Layout(
    template=template,
    title={'text': 'Healthexp dataset'},
    xaxis={'title': 'Life expectancy'},
    yaxis={
        'title': 'Country',
        'autorange': 'reversed'  # y軸の順序を反転
    }
)

# Figureを作成
figure = go.Figure(traces, layout)

figure

In [7]:
# Plotly Expressから棒グラフのFigureを作成（標準で積み上げ棒グラフ）
figure = px.bar(
    df.query('Year%10==0'),     # 10年単位で選択
    x='Year',
    y='Spending_USD',
    color='Country',
    template=template,
    title='Healthexp dataset'
)   # 国別の医療費の積み上げ棒グラフ

figure

In [8]:
# Plotly Expressから棒グラフのFigureを作成
figure = px.bar(
    df.query('Year%10==0'),
    x='Year',
    y='Spending_USD',
    color='Country',
    barmode='group',    # 棒グラフ種類（横に並べる）
    template=template,
    title='Healthexp dataset'
)   # 国別の医療費の棒グラフ

figure

In [9]:
# Plotly Expressから横棒グラフのFigureを作成
figure = px.bar(
    df.query('Year%10==0'),
    y='Year',
    x='Spending_USD',
    color='Country',
    barmode='group',
    orientation='h',    # 棒グラフ方向（横棒グラフ）
    template=template,
    title='Healthexp dataset'
)

figure

In [10]:
pd.options.plotting.backend = 'plotly'  # PandasのグラフをPlotlyに設定

# DataFrameから棒グラフのFigureを作成
figure = df.query('Year%10==0').plot.bar(
    x='Year',
    y='Life_Expectancy',
    color='Country',
    barmode='group',
    template=template,
    title='Healthexp dataset'
)   # 国別の平均余命の棒グラフ

figure

In [11]:
# Traceを作成
trace = go.Heatmap(
    x=df['Year'],           # x軸の変数
    y=df['Country'],        # z軸の変数
    z=df['Spending_USD']    # 強度の変数
)   # 医療費のヒートマップ

# Layoutを作成
layout = go.Layout(
    template=template,
    title={'text': 'Healthexp dataset'},
    xaxis={'title': 'Year'},
    yaxis={'title': 'Country'}
)

# Figureを作成
figure = go.Figure(trace, layout)

figure

In [12]:
# Traceを作成
trace = go.Heatmap(
    x=df['Year'],
    y=df['Country'],
    z=df['Spending_USD'],
    colorscale='Agsunset',              # カラースケール
    customdata=df['Life_Expectancy'],   # ホバー表示に使用する変数
    hovertemplate='%{x} %{y}<br>Spending: $%{z}<br>Life expectancy: %{customdata}',
    connectgaps=True                    # 欠損値補間あり
)   # 欠損値補間した医療費のヒートマップ

# Figureを作成
figure = go.Figure(trace, layout)

figure

In [13]:
# Traceを作成
trace = go.Heatmap(
    x=df['Year'],
    y=df['Country'],
    z=df['Spending_USD'],
    text=df['Spending_USD'],        # テキスト表示に使用する変数
    texttemplate='%{text:.1f}',     # テキスト書式（小数点1桁まで表示）
    colorscale='ice',
    customdata=df['Life_Expectancy'],
    hovertemplate='%{x} %{y}<br>Spending: $%{z}<br>Life expectancy: %{customdata}',
    connectgaps=True
)

# Figureを作成
figure = go.Figure(trace, layout)

figure